In [ ]:
import asyncio
import nest_asyncio
import websockets
import json

ip = "192.168.7.2"
port=5555
data_add = "gui_data"
control_add = "gui_control"

ws_control_add = f"ws://{ip}:{port}/{control_add}"
ws_data_add = f"ws://{ip}:{port}/{data_add}"

In [ ]:
nest_asyncio.apply() # event loop needs to be nested - otherwise it conflicts with jupyter's event loop

async def send_msg_callback(ws_address,msg):
    async with websockets.connect(ws_address) as ws:
        await ws.send(json.dumps(msg))
        print(f"Sent: {msg}")

async def rec_msg_callback(ws_address):
    async with websockets.connect(ws_address) as ws:
        while True:
            msg = await ws.recv()
            print(f"Received:{msg}") # TODO filter according to msg type

async def start_listener_callback(ws_address):
    await rec_msg_callback(ws_address)

def send_msg(ws_address,msg):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(send_msg_callback(ws_address,msg))

def start_listener(ws_address):
    loop = asyncio.get_event_loop()
    loop.create_task(start_listener_callback(ws_address)) # create_task() is needed so that the listener runs in the background and prints messages as received without blocking the cell

### connect to control ws

In [ ]:
start_listener(ws_control_add) # this gets a "connection" json response each time, so run only once or TODO filter it out


In [ ]:
send_msg(ws_control_add,  {"watcher":[{"cmd":"hi"}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "unwatch", "watchers":['myvar2']}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd":"list"}]})

### connect to data websockets  

In [ ]:
start_listener(ws_data_add)

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "watch", "watchers":['myvar2']}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "unwatch", "watchers":['myvar2']}]})

### watcher class

In [ ]:
import asyncio
import nest_asyncio
import websockets
import json
import array
from collections import deque # circular buffers 

In [ ]:
class Watcher:
    
    def __init__(self, ip="192.168.7.2", port=5555, data_add="gui_data", control_add="gui_control"):
        """ Watcher class __summary__.

        Args:
            ip (str, optional): Remote address IP. Defaults to "192.168.7.2".
            port (int, optional): Remote address port. Defaults to 5555.
            data_add (str, optional): Data endpoint. Defaults to "gui_data".
            control_add (str, optional): Control endpoint. Defaults to "gui_control".
        """
        self.ip = ip
        self.port = port
        self.data_add = data_add
        self.control_add = control_add
        self.ws_control_add = f"ws://{self.ip}:{self.port}/{self.control_add}"
        self.ws_data_add = f"ws://{self.ip}:{self.port}/{self.data_add}"
        
        self.ws_control = None
        self.ws_data = None
        
        self._ctrl_listener = None
        self._data_listener = None
        
        self._watcher_vars = [] # updates every time start is called
        self._list_response_available = asyncio.Event()
        self._list_response = None
        
        
        nest_asyncio.apply() # event loop needs to be nested - otherwise it conflicts with jupyter's event loop
    
    @property
    def watcher_vars(self):
        return self._watcher_vars # updates every time start is called
    
    @property
    def watched_vars(self):
        return [var["name"] for var in self.list() if var["watched"]]
    
    @property
    def unwatched_vars(self):
        return [var["name"] for var in self.list() if var["unwatched"]]
        
    # public methods
    
    def start(self): # TODO check Bela project_name ?
        
        if self._ctrl_listener is None: # avoid duplicate listeners
            self.start_ctrl_listener()
        if self._data_listener is None:
            self.start_data_listener()
        
        # populate watcher vars so that variables received in data stream can be identified
        self._watcher_vars = [var["name"] for var in self.list()] # TODO add error handling
    
    def stop(self):  
        if self._ctrl_listener is not None:
            self._ctrl_listener.cancel()
            self._ctrl_listener = None # empty the listener 
        if self._data_listener is not None:
            self._data_listener.cancel()
            self._data_listener = None
            
    def list(self):
        async def async_list():
            if self._ctrl_listener is None: # start listener if listener is not running
                self.start_ctrl_listener()
            self.send_ctrl_msg({"watcher": [{"cmd": "list"}]})
            # Wait for the list response to be available
            await self._list_response_available.wait()
            self._list_response_available.clear()  # Reset the event for the next call      
            return self._list_response
                
        return asyncio.run(async_list())
    
    
    def send_ctrl_msg(self, msg):
        self._send_msg(self.ws_control,self.ws_control_add, msg)
    
    def start_ctrl_listener(self):
        self._ctrl_listener = self._start_listener(self.ws_control,self.ws_control_add)
    
    def start_data_listener(self):
        self._data_listener = self._start_listener(self.ws_data, self.ws_data_add)

    # _private methods    
    
    ## start listener   

    def _start_listener(self,ws, ws_address):
        loop = asyncio.get_event_loop()
        listener_task = loop.create_task(self._start_listener_callback(ws,ws_address)) # create_task() is needed so that the listener runs in the background and prints messages as received without blocking the cell
        return listener_task
    
    async def _start_listener_callback(self, ws, ws_address):
        await self._rec_msg_callback(ws, ws_address)
        
    ## send message
    
    def _send_msg(self, ws, ws_address, msg):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self._send_msg_callback(ws,ws_address, msg))   
    
    async def _send_msg_callback(self,ws, ws_address, msg):
        try:
            async with websockets.connect(ws_address) as ws: # here you can use the same websocket for multiple messages -- but avoid using the same one for sending and receiving
                await ws.send(json.dumps(msg))
        except Exception as e:
            print(f"Error while sending message: {e}") 
    
    ## receive message     
    
    async def _rec_msg_callback(self, ws, ws_address):
        try:
            async with websockets.connect(ws_address) as ws:
                while True:
                    msg = await ws.recv()
                    if ws_address == self.ws_data_add:
                        self._parse_data_message(msg)
                    elif ws_address == self.ws_control_add:
                        self._parse_control_message(msg)
                    else:
                        print(msg)
        except Exception as e:
            print(f"Error while receiving message: {e}")
    
    
    def _parse_data_message(self, msg): # method overwritten by streamaer
        pass
    
    def _parse_control_message(self, msg):
        _msg = json.loads(msg)
        if "watcher" in _msg.keys() and "watchers" in _msg["watcher"].keys(): # list cmd
            self._list_response = _msg["watcher"]["watchers"]
            self._list_response_available.set()

    def __del__(self):
        self.stop() # stop websockets

In [ ]:
watcher = Watcher()

In [ ]:
watcher.list()

### streamer class

In [ ]:
class Streamer(Watcher):
    def __init__(self, ip="192.168.7.2", port=5555, data_add="gui_data", control_add="gui_control"):
        """ Streamer class __summary__.

            Args:
                ip (str, optional): Remote address IP. Defaults to "192.168.7.2".
                port (int, optional): Remote address port. Defaults to 5555.
                data_add (str, optional): Data endpoint. Defaults to "gui_data".
                control_add (str, optional): Control endpoint. Defaults to "gui_control".
        """
        
        super(Streamer, self).__init__(ip, port, data_add, control_add)
        
        self._streaming_buffer_size = 1000 
        self._streaming_buffer = None
        self._streaming_mode = None # OFF, FOREVER, N_FRAMES :: this flag prevents writing into the streaming buffer unless requested by the user using the start/stop_streaming() functions
        self._streaming_buffer_available = asyncio.Event()
        
    @property
    def streaming_buffer_size(self):
        return self._streaming_buffer_size
    
    @streaming_buffer_size.setter
    def streaming_buffer_size(self, value):
        self._streaming_buffer_size = value
        self._streaming_buffer = {var: deque(maxlen=self._streaming_buffer_size) for var in self._watcher_vars} # resize streaming buffer
    
    @property 
    def streaming_buffer(self):
        return {key: list(value) for key, value in self._streaming_buffer.items()} # convert dict of deque to dict of list
        
    def start(self):
        super(Streamer, self).start()
        self._streaming_buffer = {var: deque(maxlen=self._streaming_buffer_size) for var in self._watcher_vars} # init streaming buffer # FIX this is duplicated 
    
    def start_streaming(self, variables=[]): # stream forever until stopped
        self.start()
        self._streaming_mode = "FOREVER" 
        self.send_ctrl_msg({"watcher": [{"cmd": "watch", "watchers":variables}]})        
        
    def stop_streaming(self, variables=[]):
        self.stop()
        self._streaming_mode = "OFF"
        self.send_ctrl_msg({"watcher": [{"cmd": "unwatch", "watchers":variables}]})
        return self.streaming_buffer 

    async def async_stream_n_frames(self,variables, n_frames):
        self.start()
        self._streaming_mode= "N_FRAMES" # flag cleared in __rec_msg_callback
        self.streaming_buffer_size = n_frames # using setter to automatically resize buffer 
        self.send_ctrl_msg({"watcher": [{"cmd": "watch", "watchers":variables}]}) # FIX if a variable was already set as watched it will be streamed here too -- TODO set variables as unwatched if not requested

        # return when finished filling buffer 
        await self._streaming_buffer_available.wait()
        self._streaming_buffer_available.clear()
        self.stop_streaming(variables) # turns off listener, unwatches variables
        
        return self.streaming_buffer
        
    def stream_n_frames(self, variables=[], n_frames=1000, delay=0): # resizes the streaming buffer size to n_frames and returns it when full # FIX this blocks the cell? -- but needed in order to return the buffer
        # FIX now frames are counted as received by python -- once data comes timestamped, frames should be counted with respect to timestamps
        # TODO implement delay once data comes timestamped
        
        return asyncio.run(self.async_stream_n_frames(variables, n_frames))
    

    def _parse_data_message(self, msg):
        global _type, _channel # FIX this is not very clean
        if self._streaming_mode != "OFF":
            if len(msg) == 3:
                _channel = int(str(msg)[2])
                _type = str(msg)[4]
            elif len(msg) > 3:
                _msg = array.array(_type, msg).tolist()
                self._streaming_buffer[self._watcher_vars[_channel]].extend(_msg) # FIX this does not truncate once buffer size is reached
                if self._streaming_mode == "N_FRAMES" and all(len(self._streaming_buffer[var]) == self._streaming_buffer_size for var in self._watcher_vars):
                    self._streaming_mode = "OFF"
                    self._streaming_buffer_available.set()


In [ ]:
streamer = Streamer()

In [ ]:
streaming_task =  asyncio.create_task(streamer.stream_n_frames_coroutine(variables=["myvar","myvar2"], n_frames=3000))


In [ ]:
print(streaming.result())